In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Challenge d'amélioration de modèle CNN\n",
        "\n",
        "## BTS SIO  - Séance 2: Types de réseaux et applications\n",
        "\n",
        "Ce notebook vous guidera à travers un challenge d'amélioration d'un modèle CNN pour la classification d'images de vêtements (Fashion MNIST). Vous partirez d'un modèle de base volontairement sous-optimal et explorerez différentes stratégies pour améliorer ses performances.\n",
        "\n",
        "### Objectifs d'apprentissage:\n",
        "- Diagnostiquer les faiblesses d'un modèle de Deep Learning\n",
        "- Expérimenter avec différentes architectures et hyperparamètres\n",
        "- Appliquer des techniques d'optimisation (dropout, batch normalization, etc.)\n",
        "- Mesurer et comparer quantitativement les améliorations\n",
        "- Documenter méthodiquement les modifications et leurs impacts\n",
        "\n",
        "### Prérequis:\n",
        "- Connaissances de base en TensorFlow/Keras\n",
        "- Compréhension des principes des réseaux CNN\n",
        "- Avoir suivi la première partie du TP sur les CNN"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1. Configuration de l'environnement\n",
        "\n",
        "Commençons par importer les bibliothèques nécessaires."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import tensorflow as tf\n",
        "from tensorflow.keras.models import Sequential, load_model\n",
        "from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation\n",
        "from tensorflow.keras.optimizers import Adam, RMSprop, SGD\n",
        "from tensorflow.keras.preprocessing.image import ImageDataGenerator\n",
        "from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau\n",
        "from tensorflow.keras.datasets import fashion_mnist\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import pandas as pd\n",
        "import time\n",
        "import os\n",
        "import seaborn as sns\n",
        "from sklearn.metrics import confusion_matrix\n",
        "\n",
        "# Configuration pour reproductibilité\n",
        "np.random.seed(42)\n",
        "tf.random.set_seed(42)\n",
        "\n",
        "# Vérifier la version de TensorFlow\n",
        "print(f\"TensorFlow version: {tf.__version__}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2. Chargement du dataset Fashion MNIST\n",
        "\n",
        "Fashion MNIST est un dataset similaire au MNIST original, mais avec des images de vêtements au lieu de chiffres. C'est un excellent dataset pour tester des modèles de vision par ordinateur."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"Chargement du dataset Fashion MNIST...\")\n",
        "(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()\n",
        "\n",
        "# Normalisation et reshape pour correspondre au format attendu par le CNN\n",
        "x_train = x_train.reshape(-1, 28, 28, 1) / 255.0\n",
        "x_test = x_test.reshape(-1, 28, 28, 1) / 255.0\n",
        "\n",
        "# Noms des classes pour l'affichage\n",
        "class_names = ['T-shirt/top', 'Pantalon', 'Pull', 'Robe', 'Manteau',\n",
        "               'Sandale', 'Chemise', 'Basket', 'Sac', 'Bottine']\n",
        "\n",
        "print(f\"Forme des données d'entraînement: {x_train.shape}\")\n",
        "print(f\"Forme des données de test: {x_test.shape}\")\n",
        "print(f\"Nombre de classes: {len(class_names)}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation de quelques exemples\n",
        "\n",
        "Examinons à quoi ressemblent les images de notre dataset."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(10, 10))\n",
        "for i in range(25):\n",
        "    plt.subplot(5, 5, i+1)\n",
        "    plt.xticks([])\n",
        "    plt.yticks([])\n",
        "    plt.grid(False)\n",
        "    plt.imshow(x_train[i].reshape(28, 28), cmap='gray')\n",
        "    plt.xlabel(class_names[y_train[i]])\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. Tableau de bord des résultats\n",
        "\n",
        "Créons une classe pour suivre et comparer les performances des différents modèles que nous allons tester."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "class ModelImprovementDashboard:\n",
        "    \"\"\"Classe pour suivre et afficher les résultats des différentes améliorations\"\"\"\n",
        "    \n",
        "    def __init__(self):\n",
        "        self.results = []\n",
        "    \n",
        "    def add_result(self, model_name, metrics, notes=\"\"):\n",
        "        \"\"\"Ajoute un résultat au tableau de bord\"\"\"\n",
        "        result = {\n",
        "            'model_name': model_name,\n",
        "            'accuracy': metrics['test_accuracy'],\n",
        "            'loss': metrics['test_loss'],\n",
        "            'training_time': metrics['training_time'],\n",
        "            'epochs': metrics['epochs_completed'],\n",
        "            'notes': notes\n",
        "        }\n",
        "        self.results.append(result)\n",
        "    \n",
        "    def show_results(self):\n",
        "        \"\"\"Affiche un tableau comparatif des résultats\"\"\"\n",
        "        if not self.results:\n",
        "            print(\"Aucun résultat à afficher.\")\n",
        "            return\n",
        "        \n",
        "        # Créer un DataFrame\n",
        "        df = pd.DataFrame(self.results)\n",
        "        \n",
        "        # Trier par précision (descendant)\n",
        "        df = df.sort_values(by='accuracy', ascending=False)\n",
        "        \n",
        "        # Formater les colonnes\n",
        "        df['accuracy'] = df['accuracy'].apply(lambda x: f\"{x:.2f}%\")\n",
        "        df['loss'] = df['loss'].apply(lambda x: f\"{x:.4f}\")\n",
        "        df['training_time'] = df['training_time'].apply(lambda x: f\"{x:.2f}s\")\n",
        "        \n",
        "        print(\"\\n=== TABLEAU COMPARATIF DES MODÈLES ===\")\n",
        "        print(df)\n",
        "        \n",
        "        return df\n",
        "    \n",
        "    def plot_comparison(self):\n",
        "        \"\"\"Visualise la comparaison des modèles\"\"\"\n",
        "        if not self.results:\n",
        "            print(\"Aucun résultat à afficher.\")\n",
        "            return\n",
        "        \n",
        "        # Préparer les données\n",
        "        models = [r['model_name'] for r in self.results]\n",
        "        accuracies = [float(r['accuracy'].strip('%')) for r in self.results]\n",
        "        times = [float(r['training_time'].strip('s')) for r in self.results]\n",
        "        \n",
        "        # Créer le graphique\n",
        "        plt.figure(figsize=(12, 6))\n",
        "        \n",
        "        # Graphique de précision\n",
        "        plt.subplot(1, 2, 1)\n",
        "        bars = plt.bar(models, accuracies, color='skyblue')\n",
        "        plt.title('Comparaison des précisions')\n",
        "        plt.xlabel('Modèle')\n",
        "        plt.ylabel('Précision (%)')\n",
        "        plt.xticks(rotation=45, ha='right')\n",
        "        \n",
        "        # Ajouter les valeurs sur les barres\n",
        "        for bar in bars:\n",
        "            height = bar.get_height()\n",
        "            plt.text(bar.get_x() + bar.get_width()/2., height,\n",
        "                     f'{height:.2f}%',\n",
        "                     ha='center', va='bottom')\n",
        "        \n",
        "        # Graphique de temps d'entraînement\n",
        "        plt.subplot(1, 2, 2)\n",
        "        bars = plt.bar(models, times, color='salmon')\n",
        "        plt.title('Comparaison des temps d\\'entraînement')\n",
        "        plt.xlabel('Modèle')\n",
        "        plt.ylabel('Temps (secondes)')\n",
        "        plt.xticks(rotation=45, ha='right')\n",
        "        \n",
        "        # Ajouter les valeurs sur les barres\n",
        "        for bar in bars:\n",
        "            height = bar.get_height()\n",
        "            plt.text(bar.get_x() + bar.get_width()/2., height,\n",
        "                     f'{height:.2f}s',\n",
        "                     ha='center', va='bottom')\n",
        "        \n",
        "        plt.tight_layout()\n",
        "        plt.show()\n",
        "\n",
        "# Initialiser le tableau de bord\n",
        "dashboard = ModelImprovementDashboard()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. Fonctions d'évaluation de modèle\n",
        "\n",
        "Définissons des fonctions pour entraîner, évaluer et visualiser les modèles de manière cohérente."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def evaluate_model(model, x_train, y_train, x_test, y_test, epochs=5, batch_size=128, data_augmentation=False):\n",
        "    \"\"\"Entraîne et évalue un modèle, retourne les métriques de performance\"\"\"\n",
        "    \n",
        "    # Configuration pour l'augmentation de données (si activée)\n",
        "    if data_augmentation:\n",
        "        train_datagen = ImageDataGenerator(\n",
        "            rotation_range=10,\n",
        "            width_shift_range=0.1,\n",
        "            height_shift_range=0.1,\n",
        "            zoom_range=0.1,\n",
        "        )\n",
        "        train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)\n",
        "    \n",
        "    # Callbacks pour améliorer l'entraînement\n",
        "    callbacks = []\n",
        "    if epochs > 5:\n",
        "        callbacks = [\n",
        "            EarlyStopping(patience=5, restore_best_weights=True),\n",
        "            ReduceLROnPlateau(factor=0.2, patience=3, min_lr=0.0001)\n",
        "        ]\n",
        "    \n",
        "    # Mesure du temps d'entraînement\n",
        "    start_time = time.time()\n",
        "    \n",
        "    # Entraînement du modèle\n",
        "    if data_augmentation:\n",
        "        history = model.fit(\n",
        "            train_generator,\n",
        "            epochs=epochs,\n",
        "            steps_per_epoch=len(x_train) // batch_size,\n",
        "            validation_data=(x_test, y_test),\n",
        "            callbacks=callbacks,\n",
        "            verbose=1\n",
        "        )\n",
        "    else:\n",
        "        history = model.fit(\n",
        "            x_train, y_train,\n",
        "            batch_size=batch_size,\n",
        "            epochs=epochs,\n",
        "            validation_data=(x_test, y_test),\n",
        "            callbacks=callbacks,\n",
        "            verbose=1\n",
        "        )\n",
        "    \n",
        "    training_time = time.time() - start_time\n",
        "    \n",
        "    # Évaluation du modèle\n",
        "    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)\n",
        "    \n",
        "    # Préparer les métriques\n",
        "    metrics = {\n",
        "        'test_accuracy': test_acc * 100,\n",
        "        'test_loss': test_loss,\n",
        "        'training_time': training_time,\n",
        "        'epochs_completed': len(history.history['loss']),\n",
        "        'history': history\n",
        "    }\n",
        "    \n",
        "    return metrics\n",
        "\n",
        "def plot_training_history(history):\n",
        "    \"\"\"Visualise l'historique d'entraînement\"\"\"\n",
        "    plt.figure(figsize=(12, 5))\n",
        "    \n",
        "    # Graphique de précision\n",
        "    plt.subplot(1, 2, 1)\n",
        "    plt.plot(history.history['accuracy'], label='Entraînement')\n",
        "    plt.plot(history.history['val_accuracy'], label='Validation')\n",
        "    plt.title('Évolution de la précision')\n",
        "    plt.xlabel('Époque')\n",
        "    plt.ylabel('Précision')\n",
        "    plt.legend()\n",
        "    plt.grid(True, linestyle='--', alpha=0.6)\n",
        "    \n",
        "    # Graphique de perte\n",
        "    plt.subplot(1, 2, 2)\n",
        "    plt.plot(history.history['loss'], label='Entraînement')\n",
        "    plt.plot(history.history['val_loss'], label='Validation')\n",
        "    plt.title('Évolution de la perte')\n",
        "    plt.xlabel('Époque')\n",
        "    plt.ylabel('Perte')\n",
        "    plt.legend()\n",
        "    plt.grid(True, linestyle='--', alpha=0.6)\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "def plot_confusion_matrix(model, x_test, y_test):\n",
        "    \"\"\"Visualise la matrice de confusion du modèle\"\"\"\n",
        "    # Prédictions\n",
        "    y_pred = model.predict(x_test)\n",
        "    y_pred_classes = np.argmax(y_pred, axis=1)\n",
        "    \n",
        "    # Calculer la matrice de confusion\n",
        "    conf_mat = confusion_matrix(y_test, y_pred_classes)\n",
        "    \n",
        "    # Visualisation\n",
        "    plt.figure(figsize=(10, 8))\n",
        "    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',\n",
        "                xticklabels=class_names,\n",
        "                yticklabels=class_names)\n",
        "    plt.xlabel('Prédit')\n",
        "    plt.ylabel('Réel')\n",
        "    plt.title('Matrice de confusion')\n",
        "    plt.xticks(rotation=45, ha='right')\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "def show_misclassified_examples(model, x_test, y_test, n=10):\n",
        "    \"\"\"Affiche des exemples d'images mal classifiées\"\"\"\n",
        "    predictions = model.predict(x_test)\n",
        "    predicted_classes = np.argmax(predictions, axis=1)\n",
        "    \n",
        "    # Trouver les erreurs\n",
        "    errors = (predicted_classes != y_test)\n",
        "    error_indices = np.where(errors)[0]\n",
        "    \n",
        "    if len(error_indices) == 0:\n",
        "        print(\"Aucune erreur trouvée!\")\n",
        "        return\n",
        "    \n",
        "    # Sélectionner un échantillon d'erreurs\n",
        "    sample_size = min(n, len(error_indices))\n",
        "    sample_indices = np.random.choice(error_indices, size=sample_size, replace=False)\n",
        "    \n",
        "    # Afficher les exemples\n",
        "    plt.figure(figsize=(15, 3*sample_size//5 + 3))\n",
        "    for i, idx in enumerate(sample_indices):\n",
        "        plt.subplot(sample_size//5 + 1, 5, i+1)\n",
        "        plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')\n",
        "        plt.title(f\"Réel: {class_names[y_test[idx]]}\\nPrédit: {class_names[predicted_classes[idx]]}\")\n",
        "        plt.axis('off')\n",
        "    plt.tight_layout()\n",
        "    plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. Modèle de base (sous-optimal)\n",
        "\n",
        "Commençons par créer et évaluer un modèle CNN de base, volontairement sous-optimal, qui servira de point de référence pour nos améliorations."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def create_baseline_model():\n",
        "    \"\"\"Crée un modèle CNN de base volontairement sous-performant\"\"\"\n",
        "    model = Sequential([\n",
        "        Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)),\n",
        "        MaxPooling2D((2, 2)),\n",
        "        Flatten(),\n",
        "        Dense(16, activation='relu'),\n",
        "        Dense(10, activation='softmax')\n",
        "    ])\n",
        "    \n",
        "    model.compile(\n",
        "        optimizer=Adam(learning_rate=0.01),  # Learning rate trop élevé\n",
        "        loss='sparse_categorical_crossentropy',\n",
        "        metrics=['accuracy']\n",
        "    )\n",
        "    \n",
        "    return model\n",
        "\n",
        "# Créer et afficher le modèle de base\n",
        "baseline_model = create_baseline_model()\n",
        "baseline_model.summary()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Entraînement et évaluation du modèle de base"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"\\n--- Modèle de base ---\")\n",
        "baseline_metrics = evaluate_model(baseline_model, x_train, y_train, x_test, y_test, epochs=5)\n",
        "print(f\"Précision du modèle de base: {baseline_metrics['test_accuracy']:.2f}%\")\n",
        "print(f\"Temps d'entraînement: {baseline_metrics['training_time']:.2f} secondes\")\n",
        "\n",
        "# Visualiser l'historique d'entraînement\n",
        "plot_training_history(baseline_metrics['history'])\n",
        "\n",
        "# Ajouter au tableau de bord\n",
        "dashboard.add_result(\"Modèle de base\", baseline_metrics, \n",
        "                     \"CNN simple, peu de filtres, learning rate élevé\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Analyse des erreurs du modèle de base"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Afficher la matrice de confusion\n",
        "plot_confusion_matrix(baseline_model, x_test, y_test)\n",
        "\n",
        "# Afficher des exemples d'erreurs\n",
        "print(\"\\nExemples d'erreurs de classification du modèle de base:\")\n",
        "show_misclassified_examples(baseline_model, x_test, y_test)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### 🔍 Diagnostic du modèle de base\n",
        "\n",
        "Avant de passer aux améliorations, analysons les problèmes du modèle de base :\n",
        "\n",
        "1. **Architecture trop simple** : \n",
        "   - Seulement 8 filtres dans la couche de convolution\n",
        "   - Une seule couche de convolution\n",
        "   - Seulement 16 neurones dans la couche dense\n",
        "   \n",
        "2. **Optimisation problématique** :\n",
        "   - Taux d'apprentissage trop élevé (0.01)\n",
        "   - Pas de régularisation (dropout, etc.)\n",
        "   - Nombre d'époques potentiellement insuffisant\n",
        "   \n",
        "3. **Prétraitement minimal** :\n",
        "   - Pas d'augmentation de données\n",
        "   - Pas de normalisation batch\n",
        "\n",
        "Ces observations nous guideront dans nos tentatives d'amélioration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. Première amélioration : Architecture plus profonde\n",
        "\n",
        "Pour notre première amélioration, nous allons :\n",
        "- Augmenter le nombre de filtres\n",
        "- Ajouter une couche de convolution supplémentaire\n",
        "- Augmenter le nombre de neurones dans la couche dense\n",
        "- Réduire le taux d'apprentissage"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def create_improved_model_1():\n",
        "    \"\"\"Premier exemple d'amélioration: architecture plus profonde\"\"\"\n",
        "    model = Sequential([\n",
        "        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),\n",
        "        MaxPooling2D((2, 2)),\n",
        "        Conv2D(64, (3, 3), activation='relu'),\n",
        "        MaxPooling2D((2, 2)),\n",
        "        Flatten(),\n",
        "        Dense(128, activation='relu'),\n",
        "        Dense(10, activation='softmax')\n",
        "    ])\n",
        "    \n",
        "    model.compile(\n",
        "        optimizer=Adam(learning_rate=0.001),  # Taux d'apprentissage réduit\n",
        "        loss='sparse_categorical_crossentropy',\n",
        "        metrics=['accuracy']\n",
        "    )\n",
        "    \n",
        "    return model\n",
        "\n",
        "# Créer et afficher le modèle amélioré 1\n",
        "improved_model_1 = create_improved_model_1()\n",
        "improved_model_1.summary()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Entraînement et évaluation du modèle amélioré 1"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"\\n--- Modèle amélioré 1 ---\")\n",
        "improved_metrics_1 = evaluate_model(improved_model_1, x_train, y_train, x_test, y_test, epochs=10)\n",
        "print(f\"Précision du modèle amélioré 1: {improved_metrics_1['test_accuracy']:.2f}%\")\n",
        "print(f\"Temps d'entraînement: {improved_metrics_1['training_time']:.2f} secondes\")\n",
        "\n",
        "# Visualiser l'historique d'entraînement\n",
        "plot_training_history(improved_metrics_1['history'])\n",
        "\n",
        "# Ajouter au tableau de bord\n",
        "dashboard.add_result(\"Modèle amélioré 1\", improved_metrics_1, \n",
        "                    \"Plus de filtres, couche supplémentaire, learning rate plus bas\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Analyse des résultats du modèle amélioré 1"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Visualiser les résultats de l'amélioration\n",
        "print(\"Comparaison des modèles jusqu'à présent:\")\n",
        "dashboard.show_results()\n",
        "\n",
        "# Voir les nouvelles erreurs\n",
        "print(\"\\nExemples d'erreurs après la première amélioration:\")\n",
        "show_misclassified_examples(improved_model_1, x_test, y_test)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7. Deuxième amélioration : Régularisation et augmentation de données\n",
        "\n",
        "Pour notre deuxième amélioration, nous allons :\n",
        "- Ajouter du dropout pour éviter le surapprentissage\n",
        "- Intégrer la normalisation par batch (batch normalization)\n",
        "- Utiliser l'augmentation de données pour améliorer la généralisation\n",
        "\n",
        "### Architecture du modèle amélioré 2"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def create_improved_model_2():\n",
        "    \"\"\"Deuxième exemple d'amélioration: ajout de dropout et batch normalization\"\"\"\n",
        "    model = Sequential([\n",
        "        # Première couche de convolution avec batch normalization\n",
        "        Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),\n",
        "        BatchNormalization(),\n",
        "        Activation('relu'),\n",
        "        MaxPooling2D((2, 2)),\n",
        "        \n",
        "        # Deuxième couche de convolution avec batch normalization\n",
        "        Conv2D(64, (3, 3), padding='same'),\n",
        "        BatchNormalization(),\n",
        "        Activation('relu'),\n",
        "        MaxPooling2D((2, 2)),\n",
        "        \n",
        "        # Aplatissement\n",
        "        Flatten(),\n",
        "        \n",
        "        # Couche dense avec batch normalization et dropout\n",
        "        Dense(128),\n",
        "        BatchNormalization(),\n",
        "        Activation('relu'),\n",
        "        Dropout(0.5),  # 50% de dropout pour la régularisation\n",
        "        \n",
        "        # Couche de sortie\n",
        "        Dense(10, activation='softmax')\n",
        "    ])\n",
        "    \n",
        "    model.compile(\n",
        "        optimizer=Adam(learning_rate=0.001),\n",
        "        loss='sparse_categorical_crossentropy',\n",
        "        metrics=['accuracy']\n",
        "    )\n",
        "    \n",
        "    return model\n",
        "\n",
        "# Créer et afficher le modèle amélioré 2\n",
        "improved_model_2 = create_improved_model_2()\n",
        "improved_model_2.summary()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Entraînement avec augmentation de données\n",
        "\n",
        "Pour cette amélioration, nous allons également utiliser l'augmentation de données qui permet de générer artificiellement plus d'exemples d'entraînement en appliquant des transformations aux images existantes. Cela améliore la robustesse du modèle face aux variations qu'il pourrait rencontrer en conditions réelles."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"\\n--- Modèle amélioré 2 (avec augmentation de données) ---\")\n",
        "improved_metrics_2 = evaluate_model(improved_model_2, x_train, y_train, x_test, y_test, \n",
        "                                   epochs=15, data_augmentation=True)\n",
        "print(f\"Précision du modèle amélioré 2: {improved_metrics_2['test_accuracy']:.2f}%\")\n",
        "print(f\"Temps d'entraînement: {improved_metrics_2['training_time']:.2f} secondes\")\n",
        "\n",
        "# Visualiser l'historique d'entraînement\n",
        "plot_training_history(improved_metrics_2['history'])\n",
        "\n",
        "# Ajouter au tableau de bord\n",
        "dashboard.add_result(\"Modèle amélioré 2\", improved_metrics_2, \n",
        "                    \"Dropout, BatchNorm, augmentation de données\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Analyse des résultats du modèle amélioré 2"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Visualiser la matrice de confusion\n",
        "plot_confusion_matrix(improved_model_2, x_test, y_test)\n",
        "\n",
        "# Afficher des exemples d'erreurs\n",
        "print(\"\\nExemples d'erreurs après la deuxième amélioration:\")\n",
        "show_misclassified_examples(improved_model_2, x_test, y_test)\n",
        "\n",
        "# Comparer tous les modèles\n",
        "dashboard.show_results()\n",
        "dashboard.plot_comparison()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8. Création de votre propre modèle amélioré\n",
        "\n",
        "C'est maintenant à vous de concevoir votre propre amélioration! Vous pouvez explorer différentes architectures, techniques d'optimisation, ou combinaisons d'approches.\n",
        "\n",
        "Voici quelques pistes d'amélioration possibles:\n",
        "- Essayer différentes architectures (plus/moins de couches, filtres, etc.)\n",
        "- Expérimenter avec d'autres optimiseurs (RMSprop, SGD avec momentum, etc.)\n",
        "- Tester différentes techniques de régularisation\n",
        "- Modifier les paramètres d'augmentation de données\n",
        "- Utiliser des connexions résiduelles (comme dans les architectures ResNet)\n",
        "- Combiner les meilleures pratiques des modèles précédents"
      ]
    },
.\n",
        "        \n",
        "        # Couche de sortie\n",
        "        Dense(10, activation='softmax')\n",
        "    ])\n",
        "    \n",
        "    # Compilation\n",
        "    model.compile(\n",
        "        optimizer='adam',  # Modifiez selon vos préférences\n",
        "        loss='sparse_categorical_crossentropy',\n",
        "        metrics=['accuracy']\n",
        "    )\n",
        "    \n",
        "    return model\n",
        "\n",
        "# Si vous êtes prêt à tester votre modèle, décommentez les lignes suivantes\n",
        "#your_model = create_your_improved_model()\n",
        "#your_model.summary()"
      ]
    },"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
   